In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import SimpleITK as sitk

from src.evaluation.scores import dice
from src.resampling.utils import get_np_volume_from_sitk
from src.resampling.resampling import resample_np_binary_volume

In [ ]:
# If you use NiftyNet the output must be renamed to match the one requested for the submission
!python ../src/niftynet/rename_output.py ../data/segmentation_output/ ../data/segmentation_output_renamed

In [2]:
prediction_folder = '../data/segmentation_output_renamed'
groundtruth_folder = '../data/hecktor_nii/'
bb_filepath = '../data/bbox.csv'

In [3]:
# List of the files in the validation
prediction_files = [
            f
            for f in Path(prediction_folder).rglob('*.nii.gz')
        ]

# The list is sorted, so it will match the list of ground truth files
prediction_files.sort(key=lambda x: x.name.split('_')[0])   
 
# List of the patient_id in the validation
patient_name_predictions = [
    f.name.split('.')[0] for f in prediction_files
]


In [4]:
# List of the ground truth files
groundtruth_files = [
    f for f in Path(groundtruth_folder).rglob('*gtvt.nii.gz') if f.name.split('_')[0] in patient_name_predictions
]

In [8]:
prediction_files

[PosixPath('../data/segmentation_output_renamed/CHGJ016.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHGJ025.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHGJ030.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHGJ055.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHGJ086.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHMR004.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHMR028.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHUM006.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHUM016.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHUM019.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHUM022.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHUM029.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHUM033.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHUM065.nii.gz'),
 PosixPath('../data/segmentation_output_renamed/CHUS005.nii.gz'),
 PosixPath

In [5]:
# The bounding boxes will be used to compute the Dice score within.
bb_df = pd.read_csv(bb_filepath).set_index('PatientID')

In [6]:
# DataFrame to store the results
results_df = pd.DataFrame(columns=['PatientID', 'Dice Score'])

for f in prediction_files:
    patient_name = f.name.split('.')[0]
    gt_file = [k for k in groundtruth_files if k.name[:7] == patient_name][0]
    
    print('Evaluating patient {}'.format(patient_name))
    bb = (bb_df.loc[patient_name, 'x1'], bb_df.loc[patient_name, 'y1'], bb_df.loc[patient_name, 'z1'],
            bb_df.loc[patient_name, 'x2'], bb_df.loc[patient_name, 'y2'], bb_df.loc[patient_name, 'z2'])

    sitk_pred = sitk.ReadImage(str(f.resolve()))
    sitk_gt = sitk.ReadImage(str(gt_file.resolve()))
    # Transform from SimpleITK to numpy, otherwise the bounding boxes axis are swapped
    np_pred, px_spacing_pred, origin_pred = get_np_volume_from_sitk(sitk_pred)
    np_gt, px_spacing_gt, origin_gt = get_np_volume_from_sitk(sitk_gt)

    # Resample back to the original resolution and crop in the bounding box
    np_pred = resample_np_binary_volume(np_pred, origin_pred, px_spacing_pred, px_spacing_gt, bb)
    np_gt = resample_np_binary_volume(np_gt, origin_gt, px_spacing_gt, px_spacing_gt, bb)

    # Store the results
    results_df = results_df.append({'PatientID': patient_name, 'Dice Score': dice(np_gt, np_pred), 
                                    }, ignore_index=True)

    
    


IndexError: list index out of range

In [ ]:
results_df['Dice Score'].values